In [3]:
!pip install --upgrade --force-reinstall "numpy==1.26.4" "scikit-learn==1.4.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
  Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 22.1 MB/s eta 0:00:00
Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uninstalled threadpoolctl-3.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.2
    Uninstalling joblib-1.5.2:
      Successfully uninstalled joblib-1.5.2
  

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [2]:
df = pd.read_csv("dataset.csv")
print("Shape awal:", df.shape)
df.head()

Shape awal: (101, 41)


,applicant_name,age,gender,marital_status,occupation,job_position,job_specific_code,job_specific,education_level,tipe_pinjaman,...,debt_to_income_ratio,monthly_income,additional_income,job_tenure_years,dependents,existing_loans_count,existing_loans_total,payment_history_score,debt_to_income_ratio_num,approval_status
0,Ami Rahmawati,29,P,Duda,Pilot; airline,Supervisor,10612,Industri Penggilingan Aneka Kacang (termasuk L...,D4,Personal,...,baik,24565607,1291968,5,0,3,5526102,60,0.02,0
1,Hafshah Prabowo,27,L,Belum Kawin,Archaeologist,Karyawan (Employee),1193,Pertanian Tanaman Bunga,D4,Personal,...,baik,8620895,2014847,3,3,0,0,94,0.00,1
2,Dt. Darijan Usamah; M.Ak,23,P,Duda,Insurance account manager,Staff,72206,Penelitian Dan Pengembangan Sejarah,S3,Perusahaan,...,baik,18525802,2078667,0,1,3,4428924,95,0.02,1
3,Cornelia Halimah,30,L,Cerai Hidup,Trade union research officer,Staff,32402,Industri Mainan Anak-anak,S1,Perusahaan,...,baik,5069381,549782,4,1,0,0,61,0.00,1
4,Leo Irawan,43,L,Kawin,Loss adjuster; chartered,Supervisor,55191,Penginapan Remaja (youth Hostel),SMA/SMK,Personal,...,baik,8137119,521510,7,1,2,10672286,98,0.11,1


In [3]:
#drop kolom yang tidak relevan
cols_to_drop = [
    "applicant_name",
    "address",
    "company_address",
    "job_tenure",
    "debt_to_income_ratio"
]

for c in cols_to_drop:
  if c in df.columns:
    df  = df.drop(columns=c)

print("Shape setelah drop:", df.shape)

Shape setelah drop: (101, 36)


In [4]:
from pandas.core.arrays import categorical
#pastikan target sudah jelas

target_col = "approval_status"

if target_col not in df.columns:
  raise ValueError(f"Kolom target '{target_col}' tidak ditemukan dalam Dataset.")

y = df[target_col]
X = df.drop(columns=target_col)

# cek tipe data

numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()

print("Kolom numerik:", numeric_cols)
print("Kolom kategorikal:", categorical_cols)

Kolom numerik: ['age', 'job_specific_code', 'income', 'expenditure', 'language_profeciency', 'total_pinjaman', 'work_schedule', 'asset_total', 'kapasitas_usaha', 'jangka_waktu_kredit', 'monthly_income', 'additional_income', 'job_tenure_years', 'dependents', 'existing_loans_count', 'existing_loans_total', 'payment_history_score', 'debt_to_income_ratio_num']
Kolom kategorikal: ['gender', 'marital_status', 'occupation', 'job_position', 'job_specific', 'education_level', 'tipe_pinjaman', 'housing_situation', 'health_situation', 'family_background', 'social_network', 'financial_literacy', 'credit_history', 'transportation', 'asset', 'asset_range', 'sosial_ekonomi_status']


In [5]:
education_map = {
    "SMA/SMK": 1,
    "D3": 2,
    "S1": 3,
    "S2": 4,
    "S3": 5
}

if "education_level" in X.columns:
  X["education_score"] = X["education_level"].map(education_map).fillna(0)
  if "education_score" not in numeric_cols:
    numeric_cols.append("education_score")

job_position_map = {
    "Staff": 1,
    "Karyawan (Employee)": 1,
    "Supervisor": 2,
    "Kepala Divisi": 3,
    "Manager": 3,
    "Wakil Direktur": 4,
    "Direktur": 4,
    "Wakil Direktur Utama": 5,
    "Direktur Utama": 5
}

if "job_position" in X.columns:
  X["job_position_score"] = X["job_position"].map(job_position_map).fillna(0)
  if "job_position_score" not in numeric_cols:
    numeric_cols.append("job_position_score")

#capacity related features

if all(col in X.columns for col in ["monthly_income", "additional_income", "expenditure"]):
  X["net_income"] = (
      X["monthly_income"].astype(float)
      + X["additional_income"].astype(float)
      - X["expenditure"].astype(float)
  )

  if "net_income" not in numeric_cols:
    numeric_cols.append("net_income")

  X["expense_ratio"] = X["expenditure"] / (
      X["monthly_income"] + X["additional_income"] + 1
  )
  if "expense_ratio" not in numeric_cols:
    numeric_cols.append("expense_ratio")

if "job_tenure_years" in X.columns and "dependents" in X.columns:
  X["stability_score"] = X["job_tenure_years"] / (1 + X["dependents"])
  if "stability_score" not in numeric_cols:
    numeric_cols.append("stability_score")

if(
    "payment_history_score" in X.columns
    and "existing_loans_total" in X.columns
):
    X["capacity_index"] = (
        X["payment_history_score"] * 0.6
        - np.log1p(X["existing_loans_total"]) * 0.4
    )
    if "capacity_index" not in numeric_cols:
        numeric_cols.append("capacity_index")

#earning power index

if "education_score" in X.columns and "job_position_score" in X.columns and "job_tenure_years" in X.columns:
  X["earning_power_index"] = (
      X["education_score"] * 0.4
      + X["job_position_score"] * 0.4
      + (X["job_tenure_years"] / 5.0) * 0.2
  )
  if "earning_power_index" not in numeric_cols:
    numeric_cols.append("earning_power_index")

#update ulang categorical_cols (kalau ada penambahan numerik baru)
categorical_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()

print("Kolom numerik (setelah FE):", numeric_cols)
print("Kolom kategori (setelah FE):", categorical_cols)


Kolom numerik (setelah FE): ['age', 'job_specific_code', 'income', 'expenditure', 'language_profeciency', 'total_pinjaman', 'work_schedule', 'asset_total', 'kapasitas_usaha', 'jangka_waktu_kredit', 'monthly_income', 'additional_income', 'job_tenure_years', 'dependents', 'existing_loans_count', 'existing_loans_total', 'payment_history_score', 'debt_to_income_ratio_num', 'education_score', 'job_position_score', 'net_income', 'expense_ratio', 'stability_score', 'capacity_index', 'earning_power_index']
Kolom kategori (setelah FE): ['gender', 'marital_status', 'occupation', 'job_position', 'job_specific', 'education_level', 'tipe_pinjaman', 'housing_situation', 'health_situation', 'family_background', 'social_network', 'financial_literacy', 'credit_history', 'transportation', 'asset', 'asset_range', 'sosial_ekonomi_status']


In [6]:
#imputasi missing value

#numeric -> median

for col in numeric_cols:
  if col in X.columns:
    median_val = X[col].median()
    X[col] = X[col].fillna(median_val)

#kategori -> modus
for col in categorical_cols:
  mode_val = X[col].mode()
  if len(mode_val) > 0:
    X[col] = X[col].fillna(mode_val[0])

#encoding variabel kategorik (one-Hot)

X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print("Shape setelah encoding:", X_encoded.shape)

#SCALING FITUR NUMERIK
numeric_cols_encoded = X_encoded.select_dtypes(include=["int64", "float64"]).columns.tolist()

#pastikan kolom target tidak ikut
scaler = RobustScaler()
X_encoded[numeric_cols_encoded] = scaler.fit_transform(X_encoded[numeric_cols_encoded])


#train test split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)

Shape setelah encoding: (101, 355)
X_train: (80, 355)
X_test : (21, 355)
y_train: (80,)
y_test : (21,)


In [7]:
!pip install lazypredict
!pip install scikit-learn==1.2.2

In [8]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models


  0%|          | 0/29 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 64, number of negative: 16
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000176 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 447
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.800000 -> initscore=1.386294
[LightGBM] [Info] Start training from score 1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,1.00,1.00,1.00,1.00,0.21
BaggingClassifier,1.00,1.00,1.00,1.00,0.08
XGBClassifier,1.00,1.00,1.00,1.00,0.36
RandomForestClassifier,1.00,1.00,1.00,1.00,0.43
LGBMClassifier,1.00,1.00,1.00,1.00,0.14
DecisionTreeClassifier,1.00,1.00,1.00,1.00,0.03
BernoulliNB,0.86,0.91,0.91,0.87,0.06
ExtraTreeClassifier,0.86,0.82,0.82,0.86,0.05
LinearDiscriminantAnalysis,0.81,0.79,0.79,0.82,0.05
